In [4]:
import json
from typing import Dict
import numpy as np
import random

def decode_value(value: str, base: str) -> int:
    """Decode Y value based on the given base."""
    try:
        return {
            "10": lambda v: int(v),
            "2": lambda v: int(v, 2),
            "16": lambda v: int(v, 16),
            "12": lambda v: int(v, 12),
            "11": lambda v: int(v, 11),
            "14": lambda v: int(v, 14),
            "9": lambda v: int(v, 9),
            "8": lambda v: int(v, 8),
            "4": lambda v: int(v, 4)
        }[base](value)
    except KeyError:
        raise ValueError(f"Unsupported base: {base}")

def gaussian_elimination(matrix: np.ndarray) -> np.ndarray:
    """Perform Gaussian elimination."""
    n = len(matrix)
    
    for i in range(n):
        # Search for maximum in this column
        max_el = abs(matrix[i][i])
        max_row = i
        for k in range(i+1, n):
            if abs(matrix[k][i]) > max_el:
                max_el = abs(matrix[k][i])
                max_row = k

        # Swap maximum row with current row
        matrix[[i, max_row]] = matrix[[max_row, i]]

        # Make all rows below this one 0 in current column
        for k in range(i+1, n):
            c = -matrix[k][i] / matrix[i][i]
            matrix[k, i:] += c * matrix[i, i:]

    # Solve equation Ax=b for an upper triangular matrix A
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = matrix[i][-1] / matrix[i][i]
        for k in range(i-1, -1, -1):
            matrix[k][-1] -= matrix[k][i] * x[i]
    
    return x

def find_constant_term(n: int, k: int, roots: Dict[str, Dict[str, str]]) -> float:
    """Find the constant term of the polynomial using Gaussian elimination."""
    # Randomly select k roots from the available n roots
    selected_keys = random.sample(list(roots.keys()), k)
    matrix = []

    # Generate matrix rows based on the selected roots
    for key in selected_keys:
        x = int(key)  # The x-coordinate
        y = decode_value(roots[key]["value"], roots[key]["base"])  # The y-coordinate
        
        row = [x**i for i in range(k)]  # Generate the polynomial terms: [1, x, x^2, ...]
        row.append(y)  # Append the y-value (constant)
        matrix.append(row)

    # Ensure the matrix has exactly k rows and k+1 columns
    matrix = np.array(matrix, dtype=float)
    if matrix.shape[0] != k or matrix.shape[1] != k+1:
        raise ValueError(f"Incorrect matrix size: expected {k}x{k+1}, got {matrix.shape}")

    # Perform Gaussian elimination to solve the system
    x = gaussian_elimination(matrix)

    return round(x[0], 9)  # Return the constant term (f(0))

def test_case_1():
    """Handle the first test case with 'input.json'."""
    try:
        with open('input.json') as f:
            data = json.load(f)

        # Extract n, k, and roots
        n = int(data['keys']['n'])
        k = int(data['keys']['k'])
        roots = {key: value for key, value in data.items() if key not in ['keys']}

        # Find constant term
        constant_term = find_constant_term(n, k, roots)

        # Print result
        print(f"The constant term for 'input.json' is: {constant_term}")
    except FileNotFoundError:
        print("Error: input.json file not found.")
    except json.JSONDecodeError:
        print("Error: Invalid JSON format in input.json.")
    except KeyError as e:
        print(f"Error: Missing key in input.json data. {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

def test_case_2():
    """Handle the second test case with 'Secondinput.json'."""
    try:
        with open('Secondinput.json') as f:
            data = json.load(f)

        # Extract n, k, and roots
        n = int(data['keys']['n'])
        k = int(data['keys']['k'])
        roots = {key: value for key, value in data.items() if key not in ['keys']}

        # Find constant term
        constant_term = find_constant_term(n, k, roots)

        # Print result
        print(f"The constant term for 'Secondinput.json' is: {constant_term}")
    except FileNotFoundError:
        print("Error: Secondinput.json file not found.")
    except json.JSONDecodeError:
        print("Error: Invalid JSON format in Secondinput.json.")
    except KeyError as e:
        print(f"Error: Missing key in Secondinput.json data. {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

if __name__ == "__main__":
    test_case_1()
    test_case_2()


The constant term for 'input.json' is: 3.0
The constant term for 'Secondinput.json' is: 28735619723864.0
